In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pandas.plotting import scatter_matrix
from matplotlib import pyplot as plt
from sklearn import tree
#import pydotplus
import collections
from IPython.display import Image  
#import pydotplus
from sklearn.tree import export_graphviz
import random

In [2]:
import copy

In [3]:
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

#from sklearn.ensemble import RandomForestClassfier
from sklearn.feature_selection import SelectFromModel

In [4]:
df = pd.read_stata("Econ_484_data_final.dta")

In [5]:
df = df.drop("DAYSWAIT_CHRON", axis=1)

In [6]:
df = df.dropna(axis=0, how='any')

In [7]:
df['year'] = pd.DatetimeIndex(df["TX_DATE"]).year
df['month'] = pd.DatetimeIndex(df["TX_DATE"]).month
df = df.drop("TX_DATE", axis=1)

### Here I sort the dataframe by the Wait List length and put people into quintile buckets 0-4, 0 being the shortest quintile waitlist time and 4 being the longest wl time. I also create an ID indictator so I can identify people later on. 

In [8]:
 df['quint']=df.groupby('PERM_STATE')['DAYSWAIT_CHRON_KI'].transform(pd.qcut,5,labels=False)

In [9]:
df['id'] = df.index

### Now I am creating lists of the of the ID values for each quintile and then creating a list of 25 random people from our quintile sample. I can drop the quint column after I am done with this. 

In [10]:
rand_people_0_wl = df.query('quint == 0')['id'].tolist() 
sampling_0 = random.sample(rand_people_0_wl, k=25)
rand_people_1_wl = df.query('quint == 1')['id'].tolist()
sampling_1 = random.sample(rand_people_1_wl, k=25)
rand_people_2_wl = df.query('quint == 2')['id'].tolist()
sampling_2 = random.sample(rand_people_2_wl, k=25)
rand_people_3_wl = df.query('quint == 3')['id'].tolist()
sampling_3 = random.sample(rand_people_3_wl, k=25)
rand_people_4_wl = df.query('quint == 4')['id'].tolist()
sampling_4 = random.sample(rand_people_4_wl, k=25)

In [11]:
df = df.drop(['quint','id'], axis=1)

### These are the "Average" People Logan Created, I am adding them into the bottom of the data frame

In [12]:
df_ave = pd.read_excel('data_5.xlsx', sheet_name= 'Sheet2')
df_ave

,NUM_PREV_TX,A1,A2,B1,B2,DR1,DR2,gender,abo,PERM_STATE,...,PX_STAT,SHARE_TY,pstatus,ptime,AGE_GROUP,malig,LT_ONE_WEEK_DON,RECOV_OUT_US,year,month
0,0,2,24,27,49,4,11,M,A,CA,...,A,3,0,2031,A,N,N,N,2006,7
1,0,2,24,35,50,7,11,M,O,CA,...,A,3,0,2177,A,N,N,N,2006,7
2,0,2,28,35,51,7,12,M,O,CA,...,A,3,0,1677,A,N,N,N,2010,7


In [13]:
frames=[df, df_ave]
df = pd.concat(frames).reset_index(drop=True)

In [14]:
# Average People are included
df.tail()


,NUM_PREV_TX,A1,A2,B1,B2,DR1,DR2,gender,abo,PERM_STATE,...,PX_STAT,SHARE_TY,pstatus,ptime,AGE_GROUP,malig,LT_ONE_WEEK_DON,RECOV_OUT_US,year,month
401746,0,36,68,53,57,11,13,M,O,IL,...,A,3,0,3.0,A,N,N,,2018,12
401747,1,2,2,55,57,4,7,F,O,IN,...,A,3,0,19.0,P,N,N,,2018,10
401748,0,2,24,27,49,4,11,M,A,CA,...,A,3,0,2031.0,A,N,N,N,2006,7
401749,0,2,24,35,50,7,11,M,O,CA,...,A,3,0,2177.0,A,N,N,N,2006,7
401750,0,2,28,35,51,7,12,M,O,CA,...,A,3,0,1677.0,A,N,N,N,2010,7


### Making Dummy Variables for Everything Cateogorical 


In [15]:
dummy_columns = ['gender', 'abo', 'EXH_PERIT_ACCESS', 'EXH_VASC_ACCESS', 'PREV_TX', 'PREV_KI_TX',
                 'MALIG_TRR', 'txkid', 'ABO_DON','DON_TY', 'GENDER_DON',
                 'HOME_STATE_DON', 'ABO_MAT', 'GRF_STAT_KI', 'DWFG_KI', 'PREV_TX_ANY', 'PX_STAT',
                 'SHARE_TY', 'AGE_GROUP', 'malig', 'LT_ONE_WEEK_DON', 'RECOV_OUT_US', 'year', 'month',
                 'PERM_STATE']
funky_columns = ['TX_PROCEDUR_TY_KI']

In [16]:
df_non_dummy = df.drop(dummy_columns, axis=1)
#df_non_dummy

In [17]:
df_dummified = pd.get_dummies(df[dummy_columns])


In [18]:
df = df_non_dummy.merge(df_dummified, left_index=True, right_index=True)
df.shape

(401751, 217)

### Here is our Beautiful Random Forest Model, Need to cross validate and tune parameters. 

In [19]:
y_variable = 'DAYSWAIT_CHRON_KI'
X_variables = list(df.columns[df.columns != y_variable])
X = df[X_variables].to_numpy()
y = df[y_variable].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y)

forest = RandomForestRegressor(n_estimators=7)
model = forest.fit(X_train, y_train)

In [20]:
model.score(X_test,y_test)

0.8324332937892472

### I dont know what to do with this yet. Not sure how to fix it so we can read the feature importance graph. 

In [21]:
sel = SelectFromModel(RandomForestRegress(n_estimators = 100))
sel.fit(X_train, y_train)

NameError: name 'RandomForestRegress' is not defined

In [ ]:
def plot_feature_importances_(model):
    n_features = X_test.shape[1]
    plt.barh(np.arange(n_features),model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features),df.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

plot_feature_importances_(model)

In [ ]:
num_21 =  X[21,:].copy().reshape(1, -1)
num_21  = pd.DataFrame(num_21, columns = df[X_variables].columns)
num_21.iloc[0,:160]

### This is where the magic happens!

In [ ]:
state_names = ['AK','AL','AR','AS','AZ','CA','CO','CT','DC','DE','FL','GA',
               'GU','HI','IA','ID','IL','IN','KS','KY','LA','MA','MD','ME',
               'MI','MN','MO','MP','MS','MT','NA','NC','ND','NE','NH','NJ',
               'NM','NV','NY','OH','OK','OR','PA','PR','RI','SC','SD','TN',
               'TX','UT','VA','VI','VT','WA','WI','WV','WY','ZZ']

### This is a for loop that takes the random sample from quintile 0 and predicts their waitlist time changing the state they live in each time for all states and territories. It puts this info into a data frame, wl_times_0 and then we can take summary stats on this dataframe. W

### When I increase the sample size to 100 or 1,000 I am not quite sure what summary stats to report on that right now, maybe the distribution of waitlist variances? See how

In [ ]:
wl_times_0 = pd.DataFrame()
wl_times_0['state']= state_names


for j in sampling_0:
    List = []
    for i in range(58):
        A = np.zeros((58,), dtype=int)
        A[i]=1
            
        num_10000 = np.concatenate((X[j,:157].copy(),A), axis = None).reshape(1,-1)
        List.append(model.predict(num_10000))
        #print(model.predict(num_10000))
        
    List = np.asarray(List)  
    wl_times_0[j] = List


wl_times_0.describe()

In [ ]:
wl_times_1 = pd.DataFrame()
wl_times_1['state']= state_names


for j in sampling_1:
    List = []
    for i in range(58):
        A = np.zeros((58,), dtype=int)
        A[i]=1
            
        num_10000 = np.concatenate((X[j,:157].copy(),A), axis = None).reshape(1,-1)
        List.append(model.predict(num_10000))
        #print(model.predict(num_10000))
        
    List = np.asarray(List)  
    wl_times_1[j] = List


wl_times_1.describe()

In [ ]:
wl_times_2 = pd.DataFrame()
wl_times_2['state']= state_names


for j in sampling_2:
    List = []
    for i in range(58):
        A = np.zeros((58,), dtype=int)
        A[i]=1
            
        num_10000 = np.concatenate((X[j,:157].copy(),A), axis = None).reshape(1,-1)
        List.append(model.predict(num_10000))
        #print(model.predict(num_10000))
        
    List = np.asarray(List)  
    wl_times_2[j] = List


wl_times_2.describe()

In [ ]:
wl_times_3 = pd.DataFrame()
wl_times_3['state']= state_names


for j in sampling_3:
    List = []
    for i in range(58):
        A = np.zeros((58,), dtype=int)
        A[i]=1
            
        num_10000 = np.concatenate((X[j,:157].copy(),A), axis = None).reshape(1,-1)
        List.append(model.predict(num_10000))
        #print(model.predict(num_10000))
        
    List = np.asarray(List)  
    wl_times_3[j] = List


wl_times_3.describe()

In [ ]:
wl_times_4 = pd.DataFrame()
wl_times_4['state']= state_names


for j in sampling_4:
    List = []
    for i in range(58):
        A = np.zeros((58,), dtype=int)
        A[i]=1
            
        num_10000 = np.concatenate((X[j,:157].copy(),A), axis = None).reshape(1,-1)
        List.append(model.predict(num_10000))
        #print(model.predict(num_10000))
        
    List = np.asarray(List)  
    wl_times_4[j] = List


wl_times_4.describe()

### Now we repeat this process for the "average people"
### They are at the end of the data frame, they are ordered short, medium, long. 

In [ ]:


wl_times_ave_peeps = pd.DataFrame()
wl_times_ave_peeps['state']= state_names
ave_peep_id = [401748,401749,401750]

for j in ave_peep_id:
    List = []
    for i in range(58):
        A = np.zeros((58,), dtype=int)
        A[i]=1
            
        num_10000 = np.concatenate((X[j,:157].copy(),A), axis = None).reshape(1,-1)
        List.append(model.predict(num_10000))
        #print(model.predict(num_10000))
        
    List = np.asarray(List)  
    wl_times_ave_peeps[j] = List


wl_times_ave_peeps.describe()